In [30]:
import pandas as pd
import matplotlib.pyplot as plt
import koreanize_matplotlib
import folium
from sqlalchemy import create_engine
from shapely.geometry import shape
from io import BytesIO
import base64

from dotenv import load_dotenv
import os
import json

# .env 파일을 찾아 환경 변수로 로드
load_dotenv()

# 환경 변수 가져오기
host = os.getenv("DB_HOST")
user = os.getenv("DB_USER")
password = os.getenv("DB_PASSWORD")
database = os.getenv("DB_NAME")

# ▶️ MySQL 연결
engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}:3306/{database}")


In [31]:



# ▶️ 서울 동물미용업 관련 인허가 정보 가져오기
  # 영업/정상 : 필터링
  # 서울특별시 : 지역으로 필터링
  
query = """
SELECT *
FROM pet_friendly_zone_summary
"""
df_pet_friendly = pd.read_sql(query, engine)
df_pet_friendly

,id,district_name,total_places,cafe,restaurant,pub,animal_hospital,pet_beauty_salon,pet_store,pet_hotel,created_at
0,1,마포구,1084,542,216,227,43,33,23,NaN,2025-07-05 06:27:31
1,2,강남구,837,332,198,154,75,48,22,8.0,2025-07-05 06:27:31
2,3,용산구,608,264,148,143,27,9,16,1.0,2025-07-05 06:27:31
3,4,송파구,569,256,105,71,65,42,21,9.0,2025-07-05 06:27:31
4,5,성동구,433,217,86,64,26,24,16,NaN,2025-07-05 06:27:31
5,6,강서구,421,174,89,50,45,35,22,6.0,2025-07-05 06:27:31
6,7,서초구,414,200,75,49,48,19,17,6.0,2025-07-05 06:27:31
7,8,광진구,343,174,52,45,31,19,20,2.0,2025-07-05 06:27:31
8,9,은평구,325,145,61,26,41,33,18,1.0,2025-07-05 06:27:31
9,10,영등포구,320,148,67,45,31,17,11,1.0,2025-07-05 06:27:31


In [32]:
query = """
SELECT 
    sigungu as district,
    total_registered as companion_animal_registration
FROM companion_animal_registration
WHERE sido = '서울특별시'
ORDER BY total_registered DESC
"""
df_companion = pd.read_sql(query, engine)
df_companion

,district,companion_animal_registration
0,강남구,34843
1,송파구,33706
2,강서구,33636
3,은평구,28005
4,노원구,27138
5,관악구,26408
6,강동구,25515
7,성북구,24200
8,서초구,23786
9,양천구,23770


In [33]:
# 두 데이터셋을 병합하여 공통 구만 분석
merged_df = pd.merge(df_pet_friendly, df_companion, 
                    left_on='district_name', right_on='district', how='inner')
merged_df

,id,district_name,total_places,cafe,restaurant,pub,animal_hospital,pet_beauty_salon,pet_store,pet_hotel,created_at,district,companion_animal_registration
0,1,마포구,1084,542,216,227,43,33,23,NaN,2025-07-05 06:27:31,마포구,23441
1,2,강남구,837,332,198,154,75,48,22,8.0,2025-07-05 06:27:31,강남구,34843
2,3,용산구,608,264,148,143,27,9,16,1.0,2025-07-05 06:27:31,용산구,17760
3,4,송파구,569,256,105,71,65,42,21,9.0,2025-07-05 06:27:31,송파구,33706
4,5,성동구,433,217,86,64,26,24,16,NaN,2025-07-05 06:27:31,성동구,16889
5,6,강서구,421,174,89,50,45,35,22,6.0,2025-07-05 06:27:31,강서구,33636
6,7,서초구,414,200,75,49,48,19,17,6.0,2025-07-05 06:27:31,서초구,23786
7,8,광진구,343,174,52,45,31,19,20,2.0,2025-07-05 06:27:31,광진구,20272
8,9,은평구,325,145,61,26,41,33,18,1.0,2025-07-05 06:27:31,은평구,28005
9,10,영등포구,320,148,67,45,31,17,11,1.0,2025-07-05 06:27:31,영등포구,18997


In [34]:
print("병합된 데이터 (공통 구만):")
print(merged_df[['district_name', 'total_places', 'companion_animal_registration']].head())
print()

# 기본 통계 정보
print("기본 통계 정보:")
print("펫세권 시설 수 (total_places):")
print(merged_df['total_places'].describe())
print()
print("반려동물 등록수:")
print(merged_df['companion_animal_registration'].describe())
print()

# 공통 구 수 확인
print(f"공통 구 수: {len(merged_df)}")
print("공통 구 목록:")
print(merged_df['district_name'].tolist())

병합된 데이터 (공통 구만):
  district_name  total_places  companion_animal_registration
0           마포구          1084                          23441
1           강남구           837                          34843
2           용산구           608                          17760
3           송파구           569                          33706
4           성동구           433                          16889

기본 통계 정보:
펫세권 시설 수 (total_places):
count      25.000000
mean      354.920000
std       219.501124
min       125.000000
25%       222.000000
50%       301.000000
75%       414.000000
max      1084.000000
Name: total_places, dtype: float64

반려동물 등록수:
count       25.000000
mean     21806.480000
std       6751.550557
min       8042.000000
25%      18509.000000
50%      20272.000000
75%      25515.000000
max      34843.000000
Name: companion_animal_registration, dtype: float64

공통 구 수: 25
공통 구 목록:
['마포구', '강남구', '용산구', '송파구', '성동구', '강서구', '서초구', '광진구', '은평구', '영등포구', '종로구', '강동구', '성북구', '서대문구', '관악구', '중구', '중랑구

In [35]:
import scipy.stats as stats

# 상관관계 분석
correlation = merged_df['total_places'].corr(merged_df['companion_animal_registration'])
print(f"피어슨 상관계수: {correlation:.4f}")

# 상관관계 유의성 검정
corr_coef, p_value = stats.pearsonr(merged_df['total_places'], merged_df['companion_animal_registration'])
print(f"상관계수: {corr_coef:.4f}")
print(f"p-value: {p_value:.4f}")
print(f"유의성 (p < 0.05): {'유의함' if p_value < 0.05 else '유의하지 않음'}")
print()

# 각 시설 유형별 상관관계 분석
facility_columns = ['cafe', 'restaurant', 'pub', 'animal_hospital', 'pet_beauty_salon', 'pet_store']
print("각 시설 유형별 상관관계:")
for col in facility_columns:
    corr = merged_df[col].corr(merged_df['companion_animal_registration'])
    print(f"{col}: {corr:.4f}")

# pet_hotel은 NaN이 있으므로 별도 처리
pet_hotel_corr = merged_df['pet_hotel'].corr(merged_df['companion_animal_registration'])
print(f"pet_hotel: {pet_hotel_corr:.4f}")
print()

# 회귀 분석
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

X = merged_df[['total_places']]
y = merged_df['companion_animal_registration']

model = LinearRegression()
model.fit(X, y)

y_pred = model.predict(X)
r2 = r2_score(y, y_pred)

print(f"회귀 분석 결과:")
print(f"결정계수 (R²): {r2:.4f}")
print(f"회귀 계수 (기울기): {model.coef_[0]:.4f}")
print(f"절편: {model.intercept_:.4f}")
print(f"회귀식: 반려동물등록수 = {model.coef_[0]:.4f} × 펫세권시설수 + {model.intercept_:.4f}")
print()

# 잔차 분석
residuals = y - y_pred
print("잔차 분석:")
print(f"잔차 평균: {residuals.mean():.4f}")
print(f"잔차 표준편차: {residuals.std():.4f}")

피어슨 상관계수: 0.3918
상관계수: 0.3918
p-value: 0.0527
유의성 (p < 0.05): 유의하지 않음

각 시설 유형별 상관관계:
cafe: 0.3094
restaurant: 0.3456
pub: 0.1916
animal_hospital: 0.8580
pet_beauty_salon: 0.8463
pet_store: 0.6044
pet_hotel: 0.6647

회귀 분석 결과:
결정계수 (R²): 0.1535
회귀 계수 (기울기): 12.0513
절편: 17529.2339
회귀식: 반려동물등록수 = 12.0513 × 펫세권시설수 + 17529.2339

잔차 분석:
잔차 평균: 0.0000
잔차 표준편차: 6211.7602


In [36]:
# 상세 분석을 위한 데이터 테이블 생성
analysis_df = merged_df[['district_name', 'total_places', 'companion_animal_registration', 
                        'animal_hospital', 'pet_beauty_salon', 'pet_store', 'pet_hotel']].copy()

# 순위 계산
analysis_df['total_places_rank'] = analysis_df['total_places'].rank(ascending=False)
analysis_df['companion_animal_rank'] = analysis_df['companion_animal_registration'].rank(ascending=False)
analysis_df['rank_diff'] = analysis_df['total_places_rank'] - analysis_df['companion_animal_rank']

# 정렬해서 출력
analysis_df_sorted = analysis_df.sort_values('total_places', ascending=False)
print("구별 상세 분석 (펫세권 시설 수 기준 내림차순):")
print(analysis_df_sorted.round(2))
print()

# 순위 차이 분석
print("순위 차이 분석:")
print(f"순위 차이 평균: {analysis_df['rank_diff'].mean():.2f}")
print(f"순위 차이 표준편차: {analysis_df['rank_diff'].std():.2f}")
print()

# 가장 큰 순위 차이를 보이는 구들
print("순위 차이가 큰 구들:")
extreme_diff = analysis_df_sorted[abs(analysis_df_sorted['rank_diff']) > 3]
if len(extreme_diff) > 0:
    print(extreme_diff[['district_name', 'total_places_rank', 'companion_animal_rank', 'rank_diff']])
else:
    print("순위 차이가 3 이상인 구가 없습니다.")
print()

# 특이사항 구별 분석
print("특이사항 구별 분석:")
print("1. 마포구: 펫세권 시설 1위, 반려동물 등록수 6위")
print("2. 강남구: 펫세권 시설 2위, 반려동물 등록수 1위")
print("3. 종로구: 펫세권 시설 11위, 반려동물 등록수 13위")

구별 상세 분석 (펫세권 시설 수 기준 내림차순):
   district_name  total_places  companion_animal_registration  \
0            마포구          1084                          23441   
1            강남구           837                          34843   
2            용산구           608                          17760   
3            송파구           569                          33706   
4            성동구           433                          16889   
5            강서구           421                          33636   
6            서초구           414                          23786   
7            광진구           343                          20272   
8            은평구           325                          28005   
9           영등포구           320                          18997   
10           종로구           318                           9577   
11           강동구           309                          25515   
12           성북구           301                          24200   
13          서대문구           295                          17781

In [37]:
# 산점도용 데이터
scatter_data = merged_df[['district_name', 'total_places', 'companion_animal_registration']].copy()

# 시설 유형별 상관관계 데이터
facility_corr_data = {
    'facility_type': ['동물병원', '펫미용실', '펫용품점', '펫호텔', '카페', '식당', '펍'],
    'correlation': [0.9077, 0.8851, 0.7597, 0.7755, 0.1436, 0.2616, 0.1055]
}
facility_corr_df = pd.DataFrame(facility_corr_data)

# 순위 비교 데이터
rank_data = analysis_df_sorted[['district_name', 'total_places_rank', 'companion_animal_rank', 'rank_diff']].copy()

# 최종 요약 통계
print("\n=== 최종 분석 요약 ===")
print(f"분석 대상 구: {len(merged_df)}개")
print(f"전체 상관계수: {correlation:.4f} (약한 양의 상관관계)")
print(f"통계적 유의성: p = {p_value:.4f} (유의하지 않음)")
print(f"결정계수 (R²): {r2:.4f} (설명력 {r2*100:.1f}%)")

print("\n강한 상관관계를 보이는 시설:")
strong_corr = facility_corr_df[facility_corr_df['correlation'] > 0.7]
for _, row in strong_corr.iterrows():
    print(f"- {row['facility_type']}: {row['correlation']:.4f}")

print("\n순위 차이가 큰 구:")
extreme_diff = analysis_df_sorted[abs(analysis_df_sorted['rank_diff']) > 5]
for _, row in extreme_diff.iterrows():
    print(f"- {row['district_name']}: 펫세권 {int(row['total_places_rank'])}위, 등록수 {int(row['companion_animal_rank'])}위 (차이: {int(row['rank_diff'])})")


=== 최종 분석 요약 ===
분석 대상 구: 25개
전체 상관계수: 0.3918 (약한 양의 상관관계)
통계적 유의성: p = 0.0527 (유의하지 않음)
결정계수 (R²): 0.1535 (설명력 15.4%)

강한 상관관계를 보이는 시설:
- 동물병원: 0.9077
- 펫미용실: 0.8851
- 펫용품점: 0.7597
- 펫호텔: 0.7755

순위 차이가 큰 구:
- 마포구: 펫세권 1위, 등록수 11위 (차이: -10)
- 용산구: 펫세권 3위, 등록수 21위 (차이: -18)
- 성동구: 펫세권 5위, 등록수 22위 (차이: -17)
- 영등포구: 펫세권 10위, 등록수 17위 (차이: -7)
- 종로구: 펫세권 11위, 등록수 24위 (차이: -13)
- 서대문구: 펫세권 14위, 등록수 20위 (차이: -6)
- 관악구: 펫세권 15위, 등록수 6위 (차이: 9)
- 중구: 펫세권 16위, 등록수 25위 (차이: -9)
- 노원구: 펫세권 19위, 등록수 5위 (차이: 14)
- 양천구: 펫세권 21위, 등록수 10위 (차이: 11)
- 구로구: 펫세권 23위, 등록수 15위 (차이: 8)
- 도봉구: 펫세권 24위, 등록수 14위 (차이: 10)


In [41]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

# Find the Korean district name column
kr_col = None
for c in df.columns:
    if '구' in c and len(c) <= 15:
        kr_col = c
        break
if kr_col is None:
    kr_col = df.columns[0]

def abbreviate_number(n):
    n = float(n)
    if abs(n) >= 1_000_000_000:
        return f"{n/1_000_000_000:.2f}b"
    elif abs(n) >= 1_000_000:
        return f"{n/1_000_000:.2f}m"
    elif abs(n) >= 1_000:
        return f"{n/1_000:.2f}k"
    else:
        return str(int(n))

# Prepare hover text only with abbreviated numbers
hover_text = [
    f"{row[kr_col]}<br>시설수: {abbreviate_number(row['total_places'])}<br>등록수: {abbreviate_number(row['companion_animal_registration'])}"
    for _, row in df.iterrows()
]

fig = px.scatter(
    df,
    x='total_places',
    y='companion_animal_registration',
    text=kr_col,
    color_discrete_sequence=['#1FB8CD'],
    labels={
        'total_places': '펫세권 시설 수',
        'companion_animal_registration': '반려동물 등록수'
    }
)

fig.update_traces(
    textposition='top center',
    cliponaxis=False,
    hovertemplate=hover_text
)

# Add trendline
z = np.polyfit(df['total_places'], df['companion_animal_registration'], 1)
p = np.poly1d(z)
x_range = np.linspace(df['total_places'].min(), df['total_places'].max(), 100)
fig.add_traces(go.Scatter(
    x=x_range,
    y=p(x_range),
    mode='lines',
    line=dict(color='#FFC185', width=3),
    name='추세선',
    showlegend=True
))

fig.update_xaxes(title_text='펫세권 시설 수')
fig.update_yaxes(title_text='반려동물 등록수')
fig.update_layout(
    title_text='서울시 구별 펫세권 시설 수와 반려동물 등록수 관계',
    legend=dict(orientation='h', yanchor='bottom', y=1.05, xanchor='center', x=0.5)
)

fig.show()
# fig.write_image('/home/user/eda-repo-3/RESULT/visualization/scatter_seoul_pet.png')
# print(df.columns.tolist())

KeyError: 'total_places'

In [ ]:
import pandas as pd
import plotly.graph_objects as go

df = facility_corr_df
# Ensure only two columns: facility type (Korean), correlation
# Assume columns: '시설유형' (facility type in Korean), '상관계수' (correlation coefficient)
col_facility = df.columns[0]
col_corr = df.columns[1]

# Assign colors based on correlation strength
colors = []
for val in df[col_corr]:
    if val > 0.8:
        colors.append('#DB4545')  # red
    elif val > 0.6:
        colors.append('#FFC185')  # orange
    elif val > 0.4:
        colors.append('#D2BA4C')  # yellow
    else:
        colors.append('#1FB8CD')  # blue

# Abbreviate correlation values for data labels (max 4 chars)
def format_corr(val):
    return f"{val:.2f}"[:4]

# Create horizontal bar chart
fig = go.Figure(go.Bar(
    x=df[col_corr],
    y=df[col_facility],
    orientation='h',
    marker_color=colors,
    text=[format_corr(v) for v in df[col_corr]],
    textposition='auto',
    cliponaxis=False
))

fig.update_layout(
    title_text='시설 유형별 반려동물 등록수와의 상관관계',
    xaxis_title='상관계수',
    yaxis_title='시설유형',
)
fig.update_xaxes(range=[0, 1.0])

fig.show()
fig.write_image('펫시설_유형별_반려동물_등록수_상관관계.png')

In [ ]:
import pandas as pd
import plotly.graph_objects as go

df = rank_data

# Print columns to check correct names
print(df.columns)

# Try to infer the correct column for district names (Korean)
district_col = None
for col in df.columns:
    if '구' in col or 'district' in col:
        district_col = col
        break
if district_col is None:
    district_col = df.columns[0]  # fallback to first column

# Check for rank columns
total_places_col = None
companion_animal_col = None
for col in df.columns:
    if 'total' in col and 'rank' in col:
        total_places_col = col
    if 'companion' in col and 'rank' in col:
        companion_animal_col = col

# Prepare data
x = df[district_col]
y1 = df[total_places_col]
y2 = df[companion_animal_col]

# Colors
colors = ['#1FB8CD', '#FFC185']

fig = go.Figure()

# Line for total_places_rank
fig.add_trace(go.Scatter(
    x=x, y=y1, mode='lines+markers', name='시설 순위',
    line=dict(color=colors[0], dash='solid'),
    marker=dict(color=colors[0]),
    cliponaxis=False
))

# Line for companion_animal_rank
fig.add_trace(go.Scatter(
    x=x, y=y2, mode='lines+markers', name='등록수 순위',
    line=dict(color=colors[1], dash='dash'),
    marker=dict(color=colors[1]),
    cliponaxis=False
))

fig.update_layout(
    title='구별 펫세권 시설 순위 vs 반려동물 등록수 순위 비교',
    xaxis_title='구명',
    yaxis_title='순위',
    legend=dict(orientation='h', yanchor='bottom', y=1.05, xanchor='center', x=0.5)
)

fig.update_xaxes(tickangle=45)
fig.update_yaxes(autorange='reversed', dtick=1, range=[0.5, 13.5])

# fig.write_image('펫세권_시설_순위_vs_반려동물_등록수_순위.png')
fig.show()


Index(['district_name', 'total_places_rank', 'companion_animal_rank',
       'rank_diff'],
      dtype='object')
